In [0]:
# !pip install keras-self-attention
import json
from pprint import pprint
import numpy as np
import re
import io
import nltk
nltk.download('punkt')
import json
import nltk
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Dense, Activation
from keras.layers import LSTM, Bidirectional, Input
from keras.layers import concatenate,merge
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, concatenate, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU, AveragePooling1D, Reshape, GlobalAveragePooling1D
from keras.layers import Recurrent, InputSpec
from keras.models import Model, load_model
from keras import initializers
from keras.preprocessing.sequence import pad_sequences
import theano
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras_self_attention import SeqSelfAttention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=5a65912b8a43a11aa4f8794d4c36ea6ee5850bdf4ccb6e63fab61f73b0b0cb63
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
with open('train-v2.0.json') as data_file:
    data = json.load(data_file)

data_entity_list = []
for item in data['data']:
    for p in item['paragraphs']:
        for qq in p['qas']:
            for a in qq['answers']:
                data_entity_list.append((p['context'],qq['question'],a['text'],a['answer_start']))

with open('dev-v2.0.json') as data_file:
    data1 = json.load(data_file)

data_entity_list1 = []
for item in data1['data']:
    for p in item['paragraphs']:
        for qq in p['qas']:
            for a in qq['answers']:
                data_entity_list1.append((p['context'],qq['question'],a['text'],a['answer_start']))


In [0]:
import pandas as pd

df=pd.DataFrame(data_entity_list, columns=['context','question','answer','answer_start'])
val=pd.DataFrame(data_entity_list1, columns=['context','question','answer','answer_start'])

In [0]:
import re
df['answer_end']= 0
for i in range(len(df)):
    new=df['answer'][i].split()
    df['answer_end'][i]=(len(new)-1)+df['answer_start'][i]

val['answer_end']= 0
for i in range(len(val)):
    new=val['answer'][i].split()
    val['answer_end'][i]=(len(new)-1)+val['answer_start'][i]


In [0]:
X_train,y_train=df[['context','question']],df[['answer_start','answer_end']]
X_test,y_test=val[['context','question']],val[['answer_start','answer_end']]

In [0]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X_train['context'])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
lenght_list=[]
for l in X_train.question:
    lenght_list.append(len(l.split(' ')))
max_ques= np.max(lenght_list)

lenght_list=[]
for l in X_train.context:
    lenght_list.append(len(l.split(' ')))
max_doc= np.max(lenght_list)

In [0]:
words = vocab_size
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = max_doc

f = open('glove.6B.100d.txt','r',encoding="utf8")
embedding_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs 
f.close()

embedding_matrix = np.zeros((words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [0]:
''' 
To perform vectorization of text records, 
this function is taken from 'https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras' with necessary modifications.

Vectorize the words to their respective index and pad context to max context length and question to max question length.
Answers vectors are padded to the max context length as well.
'''
def vectorizeData(xContext, xQuestion, xAnswerBeing, xAnswerEnd, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        y_Begin =  np.zeros(len(xContext[i]))
        y_Begin[xAnswerBeing[i]] = 1
        y_End = np.zeros(len(xContext[i]))
        y_End[xAnswerEnd[i]] = 1
        X.append(x)
        Xq.append(xq)
        YBegin.append(y_Begin)
        YEnd.append(y_End)
    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post'), pad_sequences(YBegin, maxlen=context_maxlen, padding='post'), pad_sequences(YEnd, maxlen=context_maxlen, padding='post')

In [0]:
X_train,y_train=X_train.reset_index() ,y_train.reset_index()
tX, tXq, tYBegin, tYEnd = vectorizeData(X_train['context'],X_train['question'],y_train['answer_start'],y_train['answer_end'],tokenizer.word_index,max_doc,max_ques)

In [0]:
''' 
To perform vectorization of text records, 
this function is taken from 'https://github.com/wentaozhu/recurrent-attention-for-QA-SQUAD-based-on-keras' with necessary modifications.

Vectorize the words to their respective index and pad context to max context length and question to max question length.
Answers vectors are padded to the max context length as well.
'''
def vectorizeValData(xContext, xQuestion, word_index, context_maxlen, question_maxlen):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(xContext)):
        x = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a']  for w in xContext[i]]
        xq = [word_index[w] if w in tokenizer.word_index else tokenizer.word_index['a'] for w in xQuestion[i] ]
        X.append(x)
        Xq.append(xq)

    return pad_sequences(X, maxlen=context_maxlen, padding='post'), pad_sequences(Xq, maxlen=question_maxlen, padding='post')

# vX, vXq = vectorizeValData(X_test['context'],X_test['question'], tokenizer.word_index,max_doc,max_ques)
vX, vXq, vStart, vEnd = vectorizeData(X_test['context'],X_test['question'],y_test['answer_start'],y_test['answer_end'],tokenizer.word_index,max_doc,max_ques)

In [0]:
question_input = Input(shape=(question_maxlen,), dtype='int32', name='question_input')
context_input = Input(shape=(context_maxlen,), dtype='int32', name='context_input')

questionEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=question_maxlen, trainable=False)(question_input)  
contextEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         weights=[embedding_matrix], 
                         input_length=context_maxlen, trainable=False)(context_input) 

In [0]:
Q_h = Bidirectional(LSTM(100, return_sequences=True))(questionEmbd)
C_h = Bidirectional(LSTM(100, return_sequences=True))(contextEmbd)

merge1 = concatenate([Q_h, C_h], axis=1)
atten_2 = SeqSelfAttention(attention_activation='softmax')(merge1)
dropout_1 = Dropout(0.25)(atten_2)

QC_bilstm = Bidirectional(LSTM(100, return_sequences=True))(dropout_1)
merge2 = concatenate([QC_bilstm, C_h], axis=1)
dropout_2 = Dropout(0.5)(merge2)

start_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)
end_bilstm =  Bidirectional(LSTM(100, return_sequences=True))(dropout_2)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
start_token_dense = LSTM(context_maxlen, activation='softmax')(start_bilstm)
end_token_dense = LSTM(context_maxlen, activation='softmax')(end_bilstm)


In [0]:
model = Model(input=[context_input, question_input], output=[start_token_dense, end_token_dense])
adam = optimizers.Adam(lr=0.003)
model.compile(optimizer=adam, loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_input (InputLayer)     (None, 60)           0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 100)      10487700    question_input[0][0]             
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 100)     10487700    context_input[0][0]              
__________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [0]:
tX = tX[:50000]
tXq = tXq[:50000]
tYBegin = tYBegin[:50000]
tYEnd = tYEnd[:50000]
vX = vX[:5000]
vXq = vXq[:5000]
vStart = vStart[:5000]
vEnd = vEnd[:5000]

In [0]:
checkpoint2 = ModelCheckpoint('squad_bilstm_with_atten2_v1.h5', monitor='val_lstm_6_loss', verbose=1, save_best_only=True, mode='min', period=2)
checkpoint3 = EarlyStopping(monitor='val_lstm_6_loss', min_delta=0.01, patience=7, verbose=1, mode='min')
checkpoint4 = EarlyStopping(monitor='val_lstm_7_loss', min_delta=0.01, patience=7, verbose=1, mode='min')
callbacks_list = [checkpoint2, checkpoint3, checkpoint4] #, checkpoint3, checkpoint4]
EPOCHS = 10
model.fit([tX, tXq], [tYBegin,tYEnd], epochs=EPOCHS, batch_size=128, shuffle=True, 
          validation_split=0.2, \
          callbacks=callbacks_list, use_multiprocessing=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 2872s 72ms/step - loss: 10.8064 - lstm_6_loss: 5.3880 - lstm_7_loss: 5.4184 - val_loss: 10.6501 - val_lstm_6_loss: 5.2905 - val_lstm_7_loss: 5.3595
Epoch 2/10
40000/40000 [==============================] - 2840s 71ms/step - loss: 10.5721 - lstm_6_loss: 5.2810 - lstm_7_loss: 5.2911 - val_loss: 10.5761 - val_lstm_6_loss: 5.2696 - val_lstm_7_loss: 5.3065

Epoch 00002: val_lstm_6_loss improved from inf to 5.26961, saving model to squad_bilstm_with_atten2_v1.h5
Epoch 3/10
40000/40000 [==============================] - 2836s 71ms/step - loss: 10.5618 - lstm_6_loss: 5.2715 - lstm_7_loss: 5.2903 - val_loss: 10.5670 - val_lstm_6_loss: 5.2641 - val_lstm_7_loss: 5.3029
Epoch 4/10
40000/40000 [==============================] - 2824s 71ms/step - loss: 10.5261 - lstm_6_loss: 5.2449 - lstm_7_loss: 5.

KeyboardInterrupt: ignored

- Needed to stop iterations, because loss became 0. But, as we have saved model before loss goes to 0, we can use that to get predictions

In [0]:
trained_model = model.load_model('squad_bilstm_with_atten2_v1.h5')

predictions = trained_model.predict([vX, vXq], batch_size=128)
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()

In [1]:
# F1-Score calculation
from sklearn.metrics import f1_score

start_f1 = f1_score(vStart ,ansBegin, average="weighted")
end_f1 = f1_score(vEnd ,ansEnd, average="weighted")

#F1
(start_f1 + end_f1) / 2

0.603125

In [2]:
# Exact Match calculation

matched_token_count = 0

for t in range(len(ansBegin)):
  if ((ansBegin[t] == vStart[t]) and (ansEnd[t] == vEnd[t])):
    matched_token_count += 1

# EM
matched_token_count / len(ansBegin)

0.4951